# Tensorboard
___

Let's explore the built in data visualization capabilities that come with Tensorboard.

### Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./files/cancer_classification.csv')

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


### Train-Test Split

In [3]:
X = df.drop('benign_0__mal_1', axis=1).values

y = df['benign_0__mal_1'].values

In [4]:
X.shape

(569, 30)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

### Scaling the Data

In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_minmax = MinMaxScaler()

scaler_std = StandardScaler()

#### We will be using different Scalers in different Models

In [8]:
X_train_minmax = scaler_minmax.fit_transform(X_train)

X_test_minmax = scaler_minmax.transform(X_test)

# -------------------

X_train_std = scaler_std.fit_transform(X_train)

X_test_std = scaler_std.transform(X_test)

## Callbacks
___

### EarlyStopping Callback

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=25,
                           restore_best_weights=True, verbose = 1)

### TensorBoard Callback

In [12]:
from tensorflow.keras.callbacks import TensorBoard    # TensorBoard

TensorBoard is a visualization tool provided with TensorFlow.

This callback logs events for TensorBoard, including:

- Metrics summary plots
- Training graph visualization
- Activation histograms
- Sampled profiling

**ARGUMENTS**:

- **log_dir**: the path of the directory where to save the log files to be parsed by TensorBoard.

- **histogram_freq**: frequency (in epochs) at which to compute activation and weight histograms for the layers of the model. If set to 0, histograms won't be computed. Validation data (or split) must be specified for histogram visualizations.
    
- **write_graph**: whether to visualize the graph in TensorBoard. The log file can become quite large when write_graph is set to True.
    
- **write_images**: whether to write model weights to visualize as image in TensorBoard.

- **update_freq**: `'batch'` or `'epoch'` or integer. When using `'batch'`, writes the losses and metrics to TensorBoard after each batch. The same applies for `'epoch'`. If using an integer, let's say `1000`, the callback will write the metrics and losses to TensorBoard every 1000 samples. Note that writing too frequently to TensorBoard can slow down your training.

- **profile_batch**: Profile the batch to sample compute characteristics. By default, it will profile the second batch. Set profile_batch=0 to disable profiling. Must run in TensorFlow eager mode.

- **embeddings_freq**: frequency (in epochs) at which embedding layers will be visualized. If set to 0, embeddings won't be visualized.

___

If TensorFlow is installed using pip, you can use the following command to launch TensorBoard from the command line:

```sh
tensorboard --logdir=path_to_your_logs
```



In [13]:
pwd

'/home/raman/MEGA/Study/Data Science/Deep Learning'

## Creating & Training Models

In [14]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout

### Model 1 : Adam optimizer + MinMaxScaler

In [16]:
log_directory = "./logs/fit/" + "Model 1 - Adam MinMax"

board = TensorBoard(log_dir = log_directory,
                    histogram_freq = 1,
                    write_graph = True,
                    write_images = False,
                    update_freq = 'epoch',
                    profile_batch = 2, # (DEFAULT)
                    embeddings_freq = 1)

2021-10-28 12:17:33.571573: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:17:33.571631: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:17:33.572647: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [20]:
model_adam = Sequential()

model_adam.add(Dense(30, activation="relu"))
model_adam.add(Dropout(0.25))

model_adam.add(Dense(15, activation="relu"))
model_adam.add(Dropout(0.25))

model_adam.add(Dense(1, activation="sigmoid"))

model_adam.compile(optimizer="adam", loss='binary_crossentropy')

In [21]:
model_adam.fit(X_train_minmax, y_train, epochs=500,
               validation_data=(X_test_minmax, y_test),
               callbacks=[early_stop, board])

Epoch 1/500
14/14 [==============================] - 1s 16ms/step - loss: 0.6955 - val_loss: 0.6824


2021-10-28 12:18:08.099175: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:18:08.099227: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:18:08.135462: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-28 12:18:08.136417: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-28 12:18:08.137923: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Model 1 - Adam MinMax/train/plugins/profile/2021_10_28_12_18_08

2021-10-28 12:18:08.138730: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/fit/Model 1 - Adam MinMax/train/plugins/profile/2021_10_28_12_18_08/raman-HP-ENVY-x360.trace.json.gz
2021-10-28 12:18:08.140113: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Mode

Epoch 2/500
14/14 [==============================] - 0s 5ms/step - loss: 0.6707 - val_loss: 0.6657
Epoch 3/500
14/14 [==============================] - 0s 10ms/step - loss: 0.6631 - val_loss: 0.6449
Epoch 4/500
14/14 [==============================] - 0s 6ms/step - loss: 0.6345 - val_loss: 0.6163
Epoch 5/500
14/14 [==============================] - 0s 5ms/step - loss: 0.6035 - val_loss: 0.5770
Epoch 6/500
14/14 [==============================] - 0s 4ms/step - loss: 0.5731 - val_loss: 0.5306
Epoch 7/500
14/14 [==============================] - 0s 3ms/step - loss: 0.5141 - val_loss: 0.4784
Epoch 8/500
14/14 [==============================] - 0s 4ms/step - loss: 0.4723 - val_loss: 0.4268
Epoch 9/500
14/14 [==============================] - 0s 4ms/step - loss: 0.4195 - val_loss: 0.3775
Epoch 10/500
14/14 [==============================] - 0s 4ms/step - loss: 0.4008 - val_loss: 0.3369
Epoch 11/500
14/14 [==============================] - 0s 4ms/step - loss: 0.3766 - val_loss: 0.3080
Epoch 1

Epoch 84/500
14/14 [==============================] - 0s 6ms/step - loss: 0.0711 - val_loss: 0.1012
Epoch 85/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0742 - val_loss: 0.0920
Epoch 86/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0699 - val_loss: 0.0946
Epoch 87/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0817 - val_loss: 0.0952
Epoch 88/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0761 - val_loss: 0.0916
Epoch 89/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0702 - val_loss: 0.0918
Epoch 90/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0865 - val_loss: 0.1097
Epoch 91/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0851 - val_loss: 0.0938
Epoch 92/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0899 - val_loss: 0.0942
Epoch 93/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0818 - val_loss: 0.0906


In [22]:
print(log_directory)

./logs/fit/Model 1 - Adam MinMax


### Model 2 : Adam optimizer + StandardScaler

In [23]:
log_directory = "./logs/fit/" + "Model 2 - Adam Std"

board = TensorBoard(log_dir = log_directory,
                    histogram_freq = 1,
                    write_graph = True,
                    write_images = False,
                    update_freq = 'epoch',
                    profile_batch = 2, # (DEFAULT)
                    embeddings_freq = 1)

2021-10-28 12:18:39.551740: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:18:39.551811: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:18:39.552365: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [24]:
model_adam = Sequential()

model_adam.add(Dense(30, activation="relu"))
model_adam.add(Dropout(0.25))

model_adam.add(Dense(15, activation="relu"))
model_adam.add(Dropout(0.25))

model_adam.add(Dense(1, activation="sigmoid"))

model_adam.compile(optimizer="adam", loss='binary_crossentropy')

In [25]:
model_adam.fit(X_train_std, y_train, epochs=500,
               validation_data=(X_test_std, y_test),
               callbacks=[early_stop, board])

Epoch 1/500
14/14 [==============================] - 1s 16ms/step - loss: 0.8357 - val_loss: 0.6460


2021-10-28 12:18:56.959597: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:18:56.959777: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:18:57.003641: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-28 12:18:57.004530: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-28 12:18:57.005994: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Model 2 - Adam Std/train/plugins/profile/2021_10_28_12_18_57

2021-10-28 12:18:57.006817: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/fit/Model 2 - Adam Std/train/plugins/profile/2021_10_28_12_18_57/raman-HP-ENVY-x360.trace.json.gz
2021-10-28 12:18:57.008143: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Model 2 - 

Epoch 2/500
14/14 [==============================] - 0s 4ms/step - loss: 0.5878 - val_loss: 0.4652
Epoch 3/500
14/14 [==============================] - 0s 5ms/step - loss: 0.4502 - val_loss: 0.3757
Epoch 4/500
14/14 [==============================] - 0s 5ms/step - loss: 0.3867 - val_loss: 0.3166
Epoch 5/500
14/14 [==============================] - 0s 5ms/step - loss: 0.3386 - val_loss: 0.2721
Epoch 6/500
14/14 [==============================] - 0s 4ms/step - loss: 0.2895 - val_loss: 0.2383
Epoch 7/500
14/14 [==============================] - 0s 5ms/step - loss: 0.2959 - val_loss: 0.2097
Epoch 8/500
14/14 [==============================] - 0s 5ms/step - loss: 0.2337 - val_loss: 0.1857
Epoch 9/500
14/14 [==============================] - 0s 6ms/step - loss: 0.2266 - val_loss: 0.1654
Epoch 10/500
14/14 [==============================] - 0s 4ms/step - loss: 0.2043 - val_loss: 0.1493
Epoch 11/500
14/14 [==============================] - 0s 4ms/step - loss: 0.1902 - val_loss: 0.1357
Epoch 12

In [27]:
print(log_directory)

./logs/fit/Model 2 - Adam Std


### Model 3 : RMSProp optimizer + MinMaxScaler

In [28]:
log_directory = "./logs/fit/" + "Model 3 - RMSProp MinMax"

board = TensorBoard(log_dir = log_directory,
                    histogram_freq = 1,
                    write_graph = True,
                    write_images = False,
                    update_freq = 'epoch',
                    profile_batch = 2, # (DEFAULT)
                    embeddings_freq = 1)

2021-10-28 12:20:02.252063: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:20:02.252160: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:20:02.252928: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [29]:
model_rms = Sequential()

model_rms.add(Dense(30, activation="relu"))
model_rms.add(Dropout(0.25))

model_rms.add(Dense(15, activation="relu"))
model_rms.add(Dropout(0.25))

model_rms.add(Dense(1, activation="sigmoid"))

model_rms.compile(optimizer="rmsprop", loss='binary_crossentropy')

In [30]:
model_rms.fit(X_train_minmax, y_train, epochs=500,
              validation_data=(X_test_minmax, y_test),
              callbacks=[early_stop, board])

Epoch 1/500
14/14 [==============================] - 1s 16ms/step - loss: 0.6908 - val_loss: 0.6602


2021-10-28 12:20:11.843750: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:20:11.843812: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:20:11.885589: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-28 12:20:11.887032: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-28 12:20:11.889117: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Model 3 - RMSProp MinMax/train/plugins/profile/2021_10_28_12_20_11

2021-10-28 12:20:11.890154: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/fit/Model 3 - RMSProp MinMax/train/plugins/profile/2021_10_28_12_20_11/raman-HP-ENVY-x360.trace.json.gz
2021-10-28 12:20:11.892028: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fi

Epoch 2/500
14/14 [==============================] - 0s 6ms/step - loss: 0.6558 - val_loss: 0.6287
Epoch 3/500
14/14 [==============================] - 0s 5ms/step - loss: 0.6359 - val_loss: 0.5990
Epoch 4/500
14/14 [==============================] - 0s 4ms/step - loss: 0.5941 - val_loss: 0.5667
Epoch 5/500
14/14 [==============================] - 0s 4ms/step - loss: 0.5749 - val_loss: 0.5359
Epoch 6/500
14/14 [==============================] - 0s 6ms/step - loss: 0.5479 - val_loss: 0.5002
Epoch 7/500
14/14 [==============================] - 0s 5ms/step - loss: 0.5223 - val_loss: 0.4678
Epoch 8/500
14/14 [==============================] - 0s 4ms/step - loss: 0.4867 - val_loss: 0.4316
Epoch 9/500
14/14 [==============================] - 0s 4ms/step - loss: 0.4547 - val_loss: 0.3981
Epoch 10/500
14/14 [==============================] - 0s 4ms/step - loss: 0.4308 - val_loss: 0.3677
Epoch 11/500
14/14 [==============================] - 0s 14ms/step - loss: 0.4056 - val_loss: 0.3387
Epoch 1

Epoch 84/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0880 - val_loss: 0.0889
Epoch 85/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0770 - val_loss: 0.0840
Epoch 86/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0748 - val_loss: 0.1078
Epoch 87/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0775 - val_loss: 0.0791
Epoch 88/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0818 - val_loss: 0.0767
Epoch 89/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0779 - val_loss: 0.0840
Epoch 90/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0759 - val_loss: 0.0783
Epoch 91/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0826 - val_loss: 0.0784
Epoch 92/500
14/14 [==============================] - 0s 4ms/step - loss: 0.0756 - val_loss: 0.1071
Epoch 93/500
14/14 [==============================] - 0s 6ms/step - loss: 0.0724 - val_loss: 0.0735


In [31]:
print(log_directory)

./logs/fit/Model 3 - RMSProp MinMax


### Model 4 : RMSProp optimizer + StandardScaler

In [32]:
log_directory = "./logs/fit/" + "Model 4 - RMSProp Std"

board = TensorBoard(log_dir = log_directory,
                    histogram_freq = 1,
                    write_graph = True,
                    write_images = False,
                    update_freq = 'epoch',
                    profile_batch = 2, # (DEFAULT)
                    embeddings_freq = 1)

2021-10-28 12:20:38.562311: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:20:38.562404: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:20:38.563209: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [33]:
model_rms = Sequential()

model_rms.add(Dense(30, activation="relu"))
model_rms.add(Dropout(0.25))

model_rms.add(Dense(15, activation="relu"))
model_rms.add(Dropout(0.25))

model_rms.add(Dense(1, activation="sigmoid"))

model_rms.compile(optimizer="rmsprop", loss='binary_crossentropy')

In [34]:
model_rms.fit(X_train_std, y_train, epochs=500,
              validation_data=(X_test_std, y_test),
              callbacks=[early_stop, board])

Epoch 1/500
14/14 [==============================] - 1s 17ms/step - loss: 0.6380 - val_loss: 0.4337


2021-10-28 12:21:02.842364: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 12:21:02.842545: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 12:21:02.886936: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-28 12:21:02.888279: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-28 12:21:02.890264: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Model 4 - RMSProp Std/train/plugins/profile/2021_10_28_12_21_02

2021-10-28 12:21:02.891319: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/fit/Model 4 - RMSProp Std/train/plugins/profile/2021_10_28_12_21_02/raman-HP-ENVY-x360.trace.json.gz
2021-10-28 12:21:02.893211: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/fit/Mode

Epoch 2/500
14/14 [==============================] - 0s 10ms/step - loss: 0.4201 - val_loss: 0.2960
Epoch 3/500
14/14 [==============================] - 0s 5ms/step - loss: 0.3223 - val_loss: 0.2188
Epoch 4/500
14/14 [==============================] - 0s 4ms/step - loss: 0.2512 - val_loss: 0.1665
Epoch 5/500
14/14 [==============================] - 0s 6ms/step - loss: 0.2083 - val_loss: 0.1333
Epoch 6/500
14/14 [==============================] - 0s 8ms/step - loss: 0.1572 - val_loss: 0.1112
Epoch 7/500
14/14 [==============================] - 0s 12ms/step - loss: 0.1457 - val_loss: 0.0939
Epoch 8/500
14/14 [==============================] - 0s 4ms/step - loss: 0.1200 - val_loss: 0.0825
Epoch 9/500
14/14 [==============================] - 0s 5ms/step - loss: 0.1164 - val_loss: 0.0768
Epoch 10/500
14/14 [==============================] - 0s 4ms/step - loss: 0.1068 - val_loss: 0.0710
Epoch 11/500
14/14 [==============================] - 0s 16ms/step - loss: 0.0953 - val_loss: 0.0652
Epoch

In [35]:
print(log_directory)

./logs/fit/Model 4 - RMSProp Std


## Running Tensorboard

- <big>Run **`Terminal`** and navigate to **`pwd`** shown above</big><br><br>

- <big>Run the code : **`tensorboard --logdir ./logs/fit`**</big><br><br>

- <big>View the TensorBoard in a Web Browser @ **`http://localhost:6006/`**</big>

## TensorBoard Analysis Result
___

<big>After comparing the visualization on TendorBoard, Best Model = **`RMSProp Optimizer with StandardScaler`**</big>